In [1]:
!pip install pathway bokeh --quiet

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
# Install gdown to download files from Google Drive
!pip install gdown --quiet

# Extracted file ID from your Google Drive link
file_id = '1RqHF3zphAFOtYZgReDJUxEFweOiVAxqP'
output_path = 'your_dataset.csv'  # You can rename this

# Download the file from Google Drive
!gdown --id {file_id} -O {output_path} --quiet

# Load the CSV into a pandas DataFrame
import pandas as pd
df = pd.read_csv(output_path)

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(


In [4]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Make sure Timestamp is datetime type
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Sort first by Timestamp, then by SystemCodeNumber (parking location ID)
df = df.sort_values(by=['Timestamp', 'SystemCodeNumber']).reset_index(drop=True)

# Combine 'Latitude' and 'Longitude' into a single 'Location' column
df['Location'] = df['Latitude'].astype(str) + ' ' + df['Longitude'].astype(str)

In [5]:
print(len(df['Location'].unique()))
print(len(df['SystemCodeNumber'].unique()))

14
14


In [6]:
# Take a smaller slice of the full dataset (e.g. 5000 rows out of 18k)
df_small = df[["Timestamp", "Occupancy", "Capacity", "Location"]].iloc[:7000]

# Save to a new CSV file for use in Pathway
df_small.to_csv("parking_stream_small.csv", index=False)

In [7]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    Location: str

In [8]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream_small.csv", schema=ParkingSchema, input_rate=1000)

In [9]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    Time = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [10]:
df_small['Price'] = 10

In [11]:
df_small[["Location","Price"]].to_csv("updating_stream.csv", index=False)

In [12]:
class UpdatingSchema(pw.Schema):
    Location: str
    Price: int

In [13]:
# Static updating table
static_update = pw.io.csv.read(
    "updating_stream.csv",
    schema=UpdatingSchema,
    mode="static"
)

In [14]:
# Compute Output Values
result = data_with_time.join(
    static_update,
    pw.this.Location == static_update.Location,
).select(
    pw.this.Time,
    pw.this.Location,
    pw.this.Occupancy,
    pw.this.Capacity,
    pw.this.Price
).windowby(
    pw.this.Time,
    window=pw.temporal.tumbling(datetime.timedelta(hours=2)),
    instance=pw.this.Location,
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    Time = pw.this._pw_window_end,
    Location = pw.this._pw_instance,
    Occupancy_sum = pw.reducers.sum(pw.this.Occupancy),
    Capacity_sum = pw.reducers.sum(pw.this.Capacity),
    Price_sum = pw.reducers.sum(pw.this.Price),
    Count = pw.reducers.count(),
).with_columns(
    Occupancy = pw.this.Occupancy_sum / pw.this.Count,
    Capacity = pw.this.Capacity_sum / pw.this.Count,
    Price = pw.this.Price_sum / pw.this.Count,
    OutputPrice = (pw.this.Price_sum / pw.this.Count) + 10 * (pw.this.Occupancy_sum / pw.this.Capacity_sum)
).rename_columns(
    Price = pw.this.OutputPrice
).select(
    pw.this.Time,
    pw.this.Location,
    pw.this.Price
)

In [15]:
# STEP 1: Extract updated prices from result
updated_prices_clean = result.select(
    pw.this.Location,
    pw.this.Price
).rename_columns(
    NewPrice = pw.this.Price
)

# STEP 2: Join with static_update (LEFT JOIN)
joined = static_update.join(
    updated_prices_clean,
    pw.left.Location == pw.right.Location,
    how=pw.JoinMode.LEFT
)

# STEP 3: Use updated prices if present, else keep old ones
new_static_update = joined.select(
    Location = pw.left.Location,
    Price = pw.coalesce(pw.right.NewPrice, pw.left.Price)
)

static_update = new_static_update

In [20]:
pw.run()

Output()

In [21]:
# Write the result table to a CSV file
pw.io.csv.write(result, "parking_prices.csv")

In [23]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20
import pandas as pd

output_notebook()

# Read the data from the generated CSV file into a pandas DataFrame
df_bokeh = pd.read_csv("parking_prices.csv")

# Convert 'Time' column to datetime objects
df_bokeh['Time'] = pd.to_datetime(df_bokeh['Time'])

# Get unique locations and assign colors
locations = df_bokeh["Location"].unique()
colors = Category20[max(3, len(locations))] # Ensure enough colors for the number of locations

# Set up the figure
p = figure(title="Price per Parking Space Over Time",
           x_axis_type="datetime", width=950, height=500)

# Plot each location
for i, loc in enumerate(locations):
    loc_df = df_bokeh[df_bokeh["Location"] == loc].sort_values("Time")
    source = ColumnDataSource(loc_df)
    p.line("Time", "Price", source=source,
           legend_label=f"{loc}",
           line_width=2,
           color=colors[i % len(colors)])  # Wrap around colors

# Style
p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price"

show(p)